In [2]:
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import import_ipynb
from torchvision import datasets

from model.resnet import ResNet18

import json
import os

from irg import IRG

importing Jupyter notebook from irg.ipynb


In [3]:
import Ranger
from tqdm import tqdm
import numpy as np
from matplotlib import pyplot as plt
import timm
from torchinfo import summary #打印网络模型各层信息的库

In [4]:
def get_dataloader(train_dir, val_dir, batch_size):
    transform = transforms.Compose([
        transforms.RandomRotation(10),
        transforms.GaussianBlur(kernel_size=(5, 5), sigma=(0.1, 3.0)),
        transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.44127703, 0.4712498, 0.43714803], std=[0.18507297, 0.18050247, 0.16784933])
    ])
    
    transform_test = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.44127703, 0.4712498, 0.43714803], std=[0.18507297, 0.18050247, 0.16784933])
    ])

    dataset_train = datasets.ImageFolder(train_dir, transform=transform)
    dataset_test = datasets.ImageFolder(val_dir, transform=transform_test)
    with open('class.txt', 'w', encoding='utf-8') as file:
        file.write(str(dataset_train.class_to_idx))
    with open('class.json', 'w', encoding='utf-8') as file:
        file.write(json.dumps(dataset_train.class_to_idx))
    train_loader = torch.utils.data.DataLoader(dataset_train, batch_size = batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(dataset_test, batch_size = batch_size, shuffle=False)
    return train_loader,test_loader

In [25]:
def train(model, teacher_model, criterionKD, criterionCls, train_dataloader, test_dataloader, 
          device, batch_size, num_epoch, learning_rate, optim, init=False, scheduler_type='Cosine'):
    def init_xavier(m):
        if type(m) == nn.Linear:
            nn.init.xavier_normal_(m.weight)
            
    if init:
        model.apply(init_xavier)
        
    print(f'Model is on device: {next(model.parameters()).device}')
    print("teacher is on device:{}".format(next(teacher_model.parameters()).device))
    
    print('training on:', device)
    model.to(device)
    
    print("student_model is on device:{}".format(next(model.parameters()).device))
    print("teacher_model is on device:{}".format(next(teacher_model.parameters()).device))
    
    if optim == 'sgd':
        optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
    if optim == 'adam':
        optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
    if optim == 'adamW':
        optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)
    else:
        optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
    if scheduler_type == 'Cosine':
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=20,eta_min=1e-9)
    
    train_losses = []
    train_acces = []
    eval_acces = []
    best_acc = 0.0
    
    for epoch in range(1, num_epoch + 1):
        model.train()
        total_correct = 0
        total_loss = 0
        for (data, target) in tqdm(train_dataloader, desc = 'trainning'):
            data, target =data.to(device), target.to(device)
            l1_out_s,l2_out_s,l3_out_s,l4_out_s,fea_s, out_s = model(data)
            cls_loss = criterionCls(out_s, target)
            l1_out_t,l2_out_t,l3_out_t,l4_out_t,fea_t, out_t = teacher_model(data)
            kd_loss = criterionKD([l3_out_s, l4_out_s, fea_s, out_s],
                              [l3_out_t.detach(),
                               l4_out_t.detach(),
                               fea_t.detach(),
                               out_t.detach()]) * lambda_kd
            loss = cls_loss + kd_loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.data.item()
            _, pred = torch.max(out_s.data, 1)
            total_correct += torch.sum(pred == target).item()
        avg_loss = total_loss / len(train_dataloader)
        accuracy = total_correct / len(train_dataloader.dataset)
        print('Epoch: {}, train_loss: {:.4f}, train_accuracy: {}/{} ({:.0f}%)'.format(epoch,
            avg_loss, total_correct, len(train_dataloader.dataset), 100 * accuracy))
        train_acces.append(accuracy)
        train_losses.append(avg_loss)
        scheduler.step()
        
        model.eval()
        total_correct = 0
        total_loss = 0
        with torch.no_grad():
            for data, target in test_dataloader:
                data, target = data.to(device), target.to(device)
                _, _, _, _, _, out = model(data)
                loss = criterionCls(out, target)
                _,pred = torch.max(out.data, 1)
                total_correct += torch.sum(pred == target).item()
                total_loss += loss.data.item()
            avg_loss = total_loss / len(test_dataloader)
            accuracy = total_correct / len(test_dataloader.dataset)
            if accuracy > best_acc:
                best_acc = accuracy
                torch.save(model.state_dict(), file_dir + '/' + str(accuracy) + 'best.pth')
            eval_acces.append(accuracy)
            print('Epoch: {}, test_loss: {:.4f}, test_accuracy: {}/{} ({:.0f}%)\n'.format(epoch,
            avg_loss, total_correct, len(test_dataloader.dataset), 100 * accuracy))

In [13]:
if __name__ == '__main__':
    file_dir = 'kd_model'
    if not os.path.isdir(file_dir):
        os.makedirs(file_dir)

In [20]:
    learning_rate = 1e-4
    optimizer = 'adam'
    batch_size = 64
    num_epoch = 100
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    w_irg_vert=0.1
    w_irg_edge=5.0
    w_irg_tran=5.0
    lambda_kd=1.0

In [15]:
    train_dir = './data/train'
    test_dir = './data/val'
    train_dataloader, test_dataloader = get_dataloader(train_dir, test_dir, batch_size)

In [16]:
    model = ResNet18()
    model.fc = nn.Linear(model.fc.in_features, 12)
    teacher_model = torch.load('./teacherNet/best.pth')
    teacher_model.eval()
    criterionKD = IRG(w_irg_vert, w_irg_edge, w_irg_tran)
    criterionCls = nn.CrossEntropyLoss()

In [17]:
device

device(type='cuda')

In [26]:
    train(model,teacher_model,criterionKD,criterionCls,train_dataloader,test_dataloader,
         device,batch_size, num_epoch, learning_rate, optim)

Model is on device: cuda:0
teacher is on device:cuda:0
training on: cuda
student_model is on device:cuda:0
teacher_model is on device:cuda:0


trainning: 100%|██████████| 62/62 [01:44<00:00,  1.68s/it]


Epoch: 1, train_loss: 1.7463, train_accuracy: 3062/3961 (77%)
Epoch: 1, test_loss: 1.7152, test_accuracy: 881/1698 (52%)



trainning: 100%|██████████| 62/62 [01:43<00:00,  1.68s/it]


Epoch: 2, train_loss: 1.4554, train_accuracy: 3233/3961 (82%)
Epoch: 2, test_loss: 0.5655, test_accuracy: 1380/1698 (81%)



...

Epoch: 98, train_loss: 0.1024, train_accuracy: 3959/3961 (100%)
Epoch: 98, test_loss: 0.2881, test_accuracy: 1584/1698 (93%)



trainning: 100%|██████████| 62/62 [01:47<00:00,  1.74s/it]


Epoch: 99, train_loss: 0.1060, train_accuracy: 3961/3961 (100%)
Epoch: 99, test_loss: 0.2897, test_accuracy: 1583/1698 (93%)



trainning: 100%|██████████| 62/62 [01:47<00:00,  1.73s/it]


Epoch: 100, train_loss: 0.1047, train_accuracy: 3957/3961 (100%)
Epoch: 100, test_loss: 0.2897, test_accuracy: 1583/1698 (93%)



In [27]:
help(transforms.Pad)

Help on class Pad in module torchvision.transforms.transforms:

class Pad(torch.nn.modules.module.Module)
 |  Pad(padding, fill=0, padding_mode='constant')
 |  
 |  Pad the given image on all sides with the given "pad" value.
 |  If the image is torch Tensor, it is expected
 |  to have [..., H, W] shape, where ... means at most 2 leading dimensions for mode reflect and symmetric,
 |  at most 3 leading dimensions for mode edge,
 |  and an arbitrary number of leading dimensions for mode constant
 |  
 |  Args:
 |      padding (int or sequence): Padding on each border. If a single int is provided this
 |          is used to pad all borders. If sequence of length 2 is provided this is the padding
 |          on left/right and top/bottom respectively. If a sequence of length 4 is provided
 |          this is the padding for the left, top, right and bottom borders respectively.
 |  
 |          .. note::
 |              In torchscript mode padding as single int is not supported, use a sequen